In [1]:
import IGIMF
import numpy as np
import pandas as pd
import scipy.integrate as integr

In [2]:
mass_metals: float = 1e7 # [Msun]
mass_gas: float = 1e9 # [Msun]
M_pgal: float = 1e10 # [Msun]
downsizing_time: float = 10 # [yr] 
t: float = 1 # [Gyr]

In [3]:
igimf = IGIMF.IGIMF(mass_metals, mass_gas, M_pgal, downsizing_time, t)

In [4]:
resolution = 50
M_ecl_v = np.logspace(np.log10(igimf.M_ecl_min), np.log10(igimf.M_ecl_max), num=resolution)
m_v = np.logspace(np.log10(igimf.m_star_min), np.log10(igimf.m_star_max), num=resolution)

In [5]:
k_ecl, M_max, ECMF_func = igimf.ECMF()
sIMF_weighted = pd.DataFrame([igimf.stellar_IMF(M, ECMF_weight=ECMF_func(M))  for M in M_ecl_v], columns=['kstar', 'm_max', 'func'])
sIMF = pd.DataFrame([igimf.stellar_IMF(M)  for M in M_ecl_v], columns=['kstar', 'm_max', 'func'])
sIMF_func = sIMF['func'].values 

Inside normalized


TypeError: must be real number, not NoneType

In [ ]:
def IMF_3D_plot(m_v, M_ecl_v, sIMF_func):
    from mpl_toolkits import mplot3d
    import matplotlib.pyplot as plt
    
    def z_func(m,M):
        return np.reshape([[sIMF_func[i](m_j) for m_j in m] for i,val in enumerate(M)], (len(m), len(M)))
    
    def resh(x):
        return np.reshape(list(x) * len(x), (len(x),len(x)))
    
    fig = plt.figure(figsize=(10,8))
    # syntax for 3-D projection
    ax = plt.axes(projection ='3d')
    m = resh(m_v)
    M = resh(M_ecl_v).T
    xi = z_func(m_v, M_ecl_v)
    
    # plotting
    #ax.plot3D(x, y, z, 'green')
    ax.plot_surface(np.log10(m), np.log10(M), np.ma.log10(xi), cmap ='plasma', linewidth=0.25)
    ax.set_xlabel(r'stellar mass $m_{\star}$ [$\log_{10}(M_{\odot})$]', fontsize=15)
    ax.set_ylabel(r'E. cluster mass $M_{\rm ecl}$ [$\log_{10}(M_{\odot})$]', fontsize=15)
    ax.set_zlabel(r'$\xi_{\star}={\rm d}N_{\star}/ {\rm d} m$', fontsize=15)
    ax.set_title(r'stellar IMF $\xi_{\star}(m_{\star},M_{\rm ecl},Z)$', fontsize=17)
    fig.tight_layout()
    plt.show(block=False)
    plt.savefig(f'IMF_plot_3D.pdf', bbox_inches='tight')
    
    print(f'\nFor every cluster mass, integrate the stellar IMF over the stellar mass range:\n')
    print([integr.simpson(xi[i],x=m_v) for i,_ in enumerate(M_ecl_v)])
    print(f'\nFor every cluster mass, the mass of the cluster will be:\n')
    print([integr.simpson([xi[i]* m for m in m_v],x=m_v) for i,_ in enumerate(M_ecl_v)])

In [ ]:
IMF_3D_plot(m_v, M_ecl_v, sIMF_func)